In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# bms_only_minio.py — BMS (30 j @15 min) + upload MinIO (zone raw)

from pathlib import Path
import argparse, json, os, sys
import numpy as np
import pandas as pd

# === MinIO/S3 ===
def get_s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client(
        "s3",
        endpoint_url=endpoint,
        aws_access_key_id=access,
        aws_secret_access_key=secret,
        use_ssl=bool(secure),
        verify=bool(secure),
        region_name="us-east-1",
        config=Config(signature_version="s3v4"),
    )

def ensure_bucket(s3, bucket:str):
    import botocore
    try:
        s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError:
        s3.create_bucket(Bucket=bucket)

def s3_upload_file(s3, bucket, local_path:Path, key:str):
    s3.upload_file(str(local_path), bucket, key)

def main():
    # ---- Args ----
    ap = argparse.ArgumentParser()
    ap.add_argument("--start",  type=str, default="2025-03-01 00:00:00")
    ap.add_argument("--days",   type=int, default=30)
    ap.add_argument("--dtmin",  type=int, default=15)
    ap.add_argument("--seed",   type=int, default=42)
    ap.add_argument("--zoneid", type=str, default="brick:Room_101")
    ap.add_argument("--base",   type=str, default="~/DTE/jne_project/raw")

    # MinIO
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT", "http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER", "minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD", "minioadmin"))
    ap.add_argument("--bucket",   type=str, default="raw")
    ap.add_argument("--prefix",   type=str, default="jne_project/raw")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args, _ = ap.parse_known_args()
    np.random.seed(args.seed)

    # ---- Temps (UTC) ----
    periods = int(args.days * 24 * 60 / args.dtmin)
    ts = pd.date_range(pd.Timestamp(args.start, tz="UTC"),
                       periods=periods, freq=f"{args.dtmin}min")
    hour = (ts.hour.to_numpy() + ts.minute.to_numpy()/60.0).astype(float)
    wday = ts.weekday.to_numpy()
    dayn = (((ts.normalize() - ts[0].normalize()) / pd.Timedelta(days=1)).to_numpy().astype(int) + 1)
    mins = (ts.hour.to_numpy()*60 + ts.minute.to_numpy()).astype(int)

    # ---- Paramètres modèle ----
    DT_HR       = args.dtmin/60.0
    C, U        = 3.5, 0.25
    ETA_INT     = 0.05
    ETA_HVAC    = 0.60
    T_SET_DAY   = 22.0
    T_SET_NIGHT = 18.0
    DEADBAND    = 0.5
    K_HVAC      = 8.0
    K_LIGHT     = 3.0
    K_PLUG_BASE = 1.2
    K_PLUG_LVL  = {"low":1.0,"normal":1.5,"med":2.0,"high":3.0}

    # ---- Profils ----
    Text_like = (22 + 8*np.sin(2*np.pi*(hour-14)/24.0) + 2*np.sin(2*np.pi*(wday-3)/7.0)).astype(float).copy()
    Text_like[(dayn>=15) & (dayn<=21)] += 2.0
    ghi_shape = np.maximum(0, np.sin(np.pi*(hour-9)/8.0))

    # ---- Occupation ----
    in_shift = ((mins>=8*60) & (mins<=18*60) & (wday<5))
    presence = (np.random.rand(ts.size) < (0.90*in_shift + 0.10*(~in_shift))).astype(int)
    level    = np.full(ts.size, "low", dtype=object)
    mask_p   = presence.astype(bool)
    high_day = (dayn==10) & mask_p
    level[high_day] = "high"
    rest = mask_p & (~high_day)
    r = np.random.rand(rest.sum())
    level[rest] = np.where(r<0.6,"normal", np.where(r<0.9,"med","high"))

    # ---- Simulation BMS ----
    N = ts.size
    T_int_true = np.zeros(N); T_int_meas = np.zeros(N)
    hvac_state = np.zeros(N, dtype=int); lighting_st = np.zeros(N, dtype=int)
    T_set = np.zeros(N); P_hvac = np.zeros(N); P_light = np.zeros(N); P_plug = np.zeros(N); P_total = np.zeros(N)

    def gains_int(lvl): return {"low":0.2,"normal":0.4,"med":0.6,"high":1.0}.get(lvl,0.2)
    def plug_load(lvl): return K_PLUG_BASE + K_PLUG_LVL.get(lvl,1.5)

    T_int_true[0] = Text_like[0] - 2.0
    for i in range(N):
        pres = presence[i]==1; lvl = level[i]
        T_set[i] = T_SET_DAY if pres else T_SET_NIGHT
        lighting_st[i] = 1 if (pres and ghi_shape[i] < 0.3) else 0
        P_light[i]     = K_LIGHT * lighting_st[i]
        P_plug[i]      = plug_load(lvl)

        err = T_set[i] - (T_int_true[i-1] if i>0 else T_int_true[0])
        hvac_state[i]  = 1 if abs(err) > DEADBAND else 0
        hvac_effect    = ETA_HVAC * err if hvac_state[i]==1 else 0.0
        P_hvac[i]      = K_HVAC * abs(err) * hvac_state[i]

        if i>0:
            T_int_true[i] = T_int_true[i-1] + (DT_HR/C)*( U*(Text_like[i]-T_int_true[i-1]) + ETA_INT*gains_int(lvl) + hvac_effect )
        T_int_meas[i] = T_int_true[i] + np.random.normal(0, 0.05)
        P_total[i]    = max(P_hvac[i] + P_light[i] + P_plug[i], 0.0)

    # ---- Anomalies ----
    def mask_window(day:int, hhmm_start:str, hhmm_end:str):
        s = int(hhmm_start[:2])*60 + int(hhmm_start[3:5])
        e = int(hhmm_end[:2])*60 + int(hhmm_end[3:5])
        return (dayn==day) & (mins>=s) & (mins<=e)

    labels = []
    m = mask_window(26,"06:00","08:00")
    if m.any():
        P_total[m] += 0.40*P_total[m]
        T_int_meas[m] += 0.6
        labels.append(dict(start_ts=ts[m][0], end_ts=ts[m][-1],
                           variable="system", anomaly_type="A4_valve_stuck_open", severity="med"))
    m = mask_window(27,"10:00","12:00")
    if m.any():
        P_total[m] += 0.05*P_total[m]
        T_int_meas[m] += 0.4
        labels.append(dict(start_ts=ts[m][0], end_ts=ts[m][-1],
                           variable="system", anomaly_type="A5_damper_stuck_closed", severity="med"))

    # ---- Export RAW local ----
    bms = pd.DataFrame({
        "ts": ts, "zone_id": args.zoneid,
        "T_int": T_int_meas, "T_int_true": T_int_true,
        "hvac_state": hvac_state, "lighting_state": lighting_st, "T_set": T_set,
        "P_hvac": P_hvac, "P_lighting": P_light, "P_plug": P_plug, "P_total": P_total
    })
    labels_df = pd.DataFrame(labels)

    base = Path(args.base).expanduser().resolve()
    month = f"{ts[0].year}-{ts[0].month:02d}"
    out_bms  = base/"bms"/month
    out_lbl  = base/"labels"/month
    out_meta = base/"meta"/month
    for d in (out_bms, out_lbl, out_meta):
        d.mkdir(parents=True, exist_ok=True)

    bms_path = out_bms/"bms.csv"
    lbl_path = out_lbl/"labels_bms_systems.csv"
    bms.to_csv(bms_path, index=False)
    labels_df.to_csv(lbl_path, index=False)

    manifest = {
        "version": "1.0",
        "time": {"start_utc": str(ts[0]), "days": args.days, "dt_minutes": args.dtmin, "rows": int(len(bms))},
        "paths": {"base": str(base), "bms_csv": str(bms_path), "labels_csv": str(lbl_path), "month": month},
        "columns": {
            "time": "ts",
            "zone_id": "zone_id",
            "signals": [
                {"name":"T_int","unit":"Celsius","hint":"Temperature_Sensor"},
                {"name":"T_int_true","unit":"Celsius","hint":"Temperature_Sensor"},
                {"name":"hvac_state","unit":"One","hint":"Command/Status"},
                {"name":"lighting_state","unit":"One","hint":"Command/Status"},
                {"name":"T_set","unit":"Celsius","hint":"Temperature_Setpoint"},
                {"name":"P_hvac","unit":"kW","hint":"Power_Sensor"},
                {"name":"P_lighting","unit":"kW","hint":"Power_Sensor"},
                {"name":"P_plug","unit":"kW","hint":"Power_Sensor"},
                {"name":"P_total","unit":"kW","hint":"Power_Sensor"},
            ]
        },
        "rng_seed": args.seed
    }
    man_path = out_meta/"bms_manifest.json"
    man_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")

    # ---- Upload MinIO ----
    if not args.no_upload:
        try:
            s3 = get_s3_client(args.endpoint, args.access, args.secret, args.secure)
        except Exception as e:
            print("ERREUR: création client S3:", e, file=sys.stderr); sys.exit(2)

        ensure_bucket(s3, args.bucket)
        root = f"{args.prefix}".strip("/")

        key_b = f"{root}/bms/{month}/bms.csv"
        key_l = f"{root}/labels/{month}/labels_bms_systems.csv"
        key_m = f"{root}/meta/{month}/bms_manifest.json"

        try:
            s3_upload_file(s3, args.bucket, bms_path, key_b)
            s3_upload_file(s3, args.bucket, lbl_path, key_l)
            s3_upload_file(s3, args.bucket, man_path, key_m)
        except Exception as e:
            print("ERREUR: upload MinIO:", e, file=sys.stderr); sys.exit(3)

    print("OK — BMS")
    print("local:", bms_path, "|", lbl_path, "|", man_path)
    if not args.no_upload:
        print("minio:", f"s3://{args.bucket}/{args.prefix}/{{bms,labels,meta}}/{month}/...")

if __name__ == "__main__":
    main()


OK — BMS
local: /home/amina/DTE/jne_project/raw/bms/2025-03/bms.csv | /home/amina/DTE/jne_project/raw/labels/2025-03/labels_bms_systems.csv | /home/amina/DTE/jne_project/raw/meta/2025-03/bms_manifest.json
minio: s3://raw/jne_project/raw/{bms,labels,meta}/2025-03/...
